In [1]:
from dotenv import load_dotenv
import os
from langchain_groq import ChatGroq
import pandas as pd
import re

In [2]:
load_dotenv("secret.env")
api_key = os.getenv("GROQ_API_KEY")

In [3]:
# Initialize LLM
llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    groq_api_key=api_key,
    temperature=0.9,
    max_tokens=None,
    timeout=None,
    max_retries=2
)

In [4]:
# Read csv of monthly expenses
df = pd.read_csv("monthly-transactions.csv")
df['Transaction_ID']=df.index
df.head()
print(df)

        Date            Description  Amount (EUR) Transaction Type  \
0     9/1/24                 Salary       3500.00           Income   
1     9/1/24                   Rent       1200.00          Expense   
2     9/1/24       Electricity Bill         75.50          Expense   
3     9/1/24             Water Bill         30.00          Expense   
4     9/1/24       Internet Service         45.00          Expense   
..       ...                    ...           ...              ...   
254  9/15/24    Lawn Mowing Service         40.00          Expense   
255  9/20/24  Tree Trimming Service        150.00          Expense   
256  9/25/24   Pest Control Service         85.00          Expense   
257  9/30/24       GPT Subscription         24.99          Expense   
258  9/30/24            Cafe Mulino         12.99          Expense   

     Transaction_ID  
0                 0  
1                 1  
2                 2  
3                 3  
4                 4  
..              ...  
254  

In [54]:
# Prompt Engineering

from langchain_core.prompts import PromptTemplate

prompt_categorization = PromptTemplate.from_template(
    """
    You are a financial assistant. Your task is to categorize the provided transactions into appropriate categories.

    Input format:
    'Transaction ID | Transaction Name - Income/Expense'. Transactions are separated by commas.

    Categorization rules:
    1. Categorize each transaction based on its name (e.g., coffee shop → Food and Drinks, retail store → Shopping).
    2. Each transaction must be assigned to only one category.
    3. For ambiguous transactions, use "Miscellaneous Expense" or "Miscellaneous Income" as appropriate.
    4. No transaction should be left uncategorized.

    Output format:
    - Group transactions by category. Do not include numbering or bullet points.
    - Use the format: 
      Category Name:
         Transaction ID | Transaction Name
         Transaction ID | Transaction Name
    - Do not include categories with no transactions present.
    - Do not include the transaction type (Income/Expense) from the input.
    - Do not include explanations, commentary, or reclassification instructions.

    Categories:
    Expense Categories:
    - Housing and Utilities: Rent, water bill, insurance, utilities, etc.
    - Transportation: Metro, taxis, ride-sharing, etc.
    - Vehicle Maintenance: Car repairs, gas, car washes, etc.
    - Food and Drinks: Restaurants, cafes, coffee shops, etc.
    - Groceries: Supermarket, grocery store purchases.
    - Health and Wellness: Healthcare, pharmacies, etc.
    - Personal Care: Beauty, salons, spas, barbershops.
    - Shopping: Retail, online shopping (clothing, electronics, household items).
    - Entertainment: Movies, streaming services, concerts, etc.
    - Repairs and Maintenance: Repairs unrelated to vehicles (appliances, etc.).
    - Subscriptions and Fees: Recurring fees (subscriptions, memberships).
    - Miscellaneous Expense: For any uncategorized or ambiguous transactions.

    Income Categories:
    - Salary: Regular income from employment.
    - Investments: Dividends, interest, capital gains, etc.
    - Miscellaneous Income: Gifts, bonuses, or other uncategorized income.

    Transactions to categorize:
    {transactions}
    """
)


In [55]:
'''
For each row, it formats the Transaction ID (unique id for each transaction), Description, Transaction type (Expense / Income) into a single string with delimiter so that, LLM can more easily interpret them.
['1 | Rent - Expense',....]
'''

def format_transactions(transactions):
    formatted_transactions = []
    for _, row in transactions.iterrows():
        formatted_transactions.append(f"{row['Transaction_ID']} | {row['Description']} - {row['Transaction Type']}")
    print(".....formatted_transactions.....",formatted_transactions)
    return formatted_transactions


In [56]:
'''
Converts the list of formatted expenses into a single text string seperated by ',' 
expense_text: 1 | Rent - Expense, 2 | Electricity Bill - Expense,...
Sends expense_text to LLM
'''

def categorize_transactions(transaction_list):
    transaction_text = ', '.join(transaction_list)
    print(".....transaction text sent to llm.....",transaction_text)
    chain_categorization = prompt_categorization | llm
    response = chain_categorization.invoke({"transactions": transaction_text})
    return response.content
    

In [57]:
# Defining chunk size due to context limit
chunk_size=25
# Empty list to append results of each chunk batch
categorized_transactions=[]

In [58]:
'''
Loop over the batches of chunks, send each batch to LLm and concatenate the response
'''

for i in range(0, len(df), chunk_size):
    # Select the current batch of expenses
    current_transactions = df.iloc[i:i+chunk_size]
    formatted_transactions = format_transactions(current_transactions)
    categorized_response = categorize_transactions(formatted_transactions)
    categorized_transactions.append(categorized_response)
print("....categorized_expenses (final list).....",categorized_transactions)
    

.....formatted_transactions..... ['0 | Salary - Income', '1 | Rent - Expense', '2 | Electricity Bill - Expense', '3 | Water Bill - Expense', '4 | Internet Service - Expense', '5 | Mobile Phone Plan - Expense', '6 | Gym Membership - Expense', '7 | Netflix Subscription - Expense', '8 | Home Insurance - Expense', '9 | Car Insurance - Expense', '10 | Health Insurance - Expense', '11 | Spotify Premium - Expense', '12 | Café Noir - Expense', '13 | Metro Ticket - Expense', '14 | Lunch at Work Cafeteria - Expense', '15 | Supermarket Express - Expense', '16 | Bakery Deluxe - Expense', '17 | Metro Ticket - Expense', '18 | Lunch at Veggie Corner - Expense', '19 | Pharmacy - Expense', '20 | Café Noir - Expense', '21 | Metro Ticket - Expense', '22 | Lunch at Pasta Paradise - Expense', '23 | Bookstore - Expense', '24 | Bakery Deluxe - Expense']
.....transaction text sent to llm..... 0 | Salary - Income, 1 | Rent - Expense, 2 | Electricity Bill - Expense, 3 | Water Bill - Expense, 4 | Internet Servic

In [59]:
'''
Postprocessing of the response, creates a tuple with (ID, description, category) for every row, 
stores all tuples in the list.
'''
def postprocessing_response(categorized_response):
    processed_data = []
    
    # Split the categorized response by double newline to separate categories
    categories = re.split(r'\n\n', categorized_response.strip())
    
    # Iterate over each category block
    for category_block in categories:
        # Split the category name and the transactions using the colon (":")
        parts = category_block.split(':')
        
        # If no valid category or transactions, skip
        if len(parts) < 2:
            continue
        
        # Extract category name and transactions block
        category_name = parts[0].strip()  # Category name, e.g., 'Housing and Utilities'
        transactions_block = parts[1].strip()  # Transactions in this category
        
        # Find all transactions in the block, using the format Transaction_ID | Description
        # We expect the format: "Transaction_ID | Description"
        transactions = re.findall(r'(\d+)\s*\|\s*([^\n]+)', transactions_block)
        
        # For each transaction, append to processed_data as (Transaction_ID, Description, Category)
        for transaction_id, transaction_description in transactions:
            processed_data.append((transaction_id.strip(), transaction_description.strip(), category_name))
    
    return processed_data


In [60]:
all_entries = []

for text in categorized_transactions:
    all_entries.extend(postprocessing_response(text))


print("......all entries.........",all_entries)

......all entries......... [('0', 'Salary', 'Salary'), ('1', 'Rent', 'Housing and Utilities'), ('2', 'Electricity Bill', 'Housing and Utilities'), ('3', 'Water Bill', 'Housing and Utilities'), ('4', 'Internet Service', 'Housing and Utilities'), ('5', 'Home Insurance', 'Housing and Utilities'), ('8', 'Home Insurance', 'Housing and Utilities'), ('13', 'Metro Ticket', 'Transportation'), ('17', 'Metro Ticket', 'Transportation'), ('21', 'Metro Ticket', 'Transportation'), ('12', 'Café Noir', 'Food and Drinks'), ('14', 'Lunch at Work Cafeteria', 'Food and Drinks'), ('18', 'Lunch at Veggie Corner', 'Food and Drinks'), ('20', 'Café Noir', 'Food and Drinks'), ('22', 'Lunch at Pasta Paradise', 'Food and Drinks'), ('15', 'Supermarket Express', 'Groceries'), ('16', 'Bakery Deluxe', 'Groceries'), ('24', 'Bakery Deluxe', 'Groceries'), ('10', 'Health Insurance', 'Health and Wellness'), ('19', 'Pharmacy', 'Health and Wellness'), ('6', 'Gym Membership', 'Subscriptions and Fees'), ('7', 'Netflix Subscrip

In [61]:
# Create a DataFrame from the extracted data
df_expenses = pd.DataFrame(all_entries, columns=["Transaction_ID", "Description", "Category"])
# 
df_expenses['Transaction_ID'] = df_expenses['Transaction_ID'].astype(int)

In [62]:
df_expenses.to_csv('categorized_transactions(LLM response).csv',index=False)

In [63]:
final_df=pd.merge(df, df_expenses, on="Transaction_ID", how="left")
final_df.head()
print(final_df)

        Date          Description_x  Amount (EUR) Transaction Type  \
0     9/1/24                 Salary       3500.00           Income   
1     9/1/24                   Rent       1200.00          Expense   
2     9/1/24       Electricity Bill         75.50          Expense   
3     9/1/24             Water Bill         30.00          Expense   
4     9/1/24       Internet Service         45.00          Expense   
..       ...                    ...           ...              ...   
256  9/15/24    Lawn Mowing Service         40.00          Expense   
257  9/20/24  Tree Trimming Service        150.00          Expense   
258  9/25/24   Pest Control Service         85.00          Expense   
259  9/30/24       GPT Subscription         24.99          Expense   
260  9/30/24            Cafe Mulino         12.99          Expense   

     Transaction_ID          Description_y               Category  
0                 0                 Salary                 Salary  
1                 1    

In [64]:
final_df.to_csv('categorized_transactions.csv',index=False)

In [ ]:

    
    '''
    # Define mappings of similar categories to unified names (without prompt engineering)
    if "food" in category or "dining" in category or in category or "snacks" in category or "beverages" in category:
        return "Food and Drinks"
    elif "transportation" in category or "vehicle" in category:
        return "Transportation"
    elif "entertainment" in category or "recreation" in category:
        return "Entertainment"
    elif "housing" in category or "bill" in category or "utilities" in category or "home" in category:
        return "Housing and Utilities"
    elif "health" in category or "wellness" in category or "pets" in category:
        return "Health and Wellness"
    elif "shopping" in category or "miscellaneous" in category or "non-food" in category:
        return "Shopping"
    elif "finance" in category or "banking" in category or "insurance" in category or "fees" in category or "technology" in category "digital" in category:
        return "Subscription and Fees"
    elif "personal" in category or "care" in category or "services" in category or "beauty" in category:
        return "Personal Care"
    elif "office" in category or "education" in category or "school" in category:
        return "Education"
    elif "garden" in category or "home" in category or "repair" in category:
        return "Maintenance and Repair"
    else:
        return category
    '''


    